In [1]:
import gym
import time
import gym_maze
import numpy as np
from collections import deque 

In [2]:
'''
    Function: render_path()
    Description: This function moves the robot according to the specified path
    Input:
        - env: the map for the robot
        - path_list: list of moves the robot has to make ('N', 'E', 'S', or 'W')
    Output: None
'''
def render_path(env, path_list, delay = 0.3):
    if type(path_list) is not list:
        path_list = list(path_list)

    for path in path_list:
        _ = env.step(path)
        _ = env.render()
        time.sleep(delay)
        

'''
    Ignore this function, it is not required for this project
'''
def move_to(env, position, delay = 0.3):
    row, col = position
    env.env.state = [col, row]
    env.env.maze_view._MazeView2D__draw_robot(transparency = 0)
    env.env.maze_view._MazeView2D__robot = [col, row]
    _ = env.render()
    time.sleep(delay)
        

'''
    Function: get_possible_path()
    Desciption: The function gets the possible moves based on the position of the robot
    Input:
        - env: the map for the robot
        - position: current position of the robot: (row, col) tuples
    Output:
        - possible_move: list of possible moves the robot can make ('N', 'E', 'S', or 'W')
        - move_position: list of (x,y) position the robot will be based on the moves made
'''
def get_possible_moves(env, position):
    '''        
        1 -> up
        2 -> right
        4 -> down
        8 -> left
    '''
    maze_size = env.env.maze_size
    
    row = position[0]
    col = position[1]
    if row < 0 or row >= maze_size[0] or col < 0 or col >= maze_size[1]:
        raise IndexError('Maze position out of bound')
       
    move_matrix = env.env.maze_view.maze.maze_cells.T
    possible_move = []
    move_position = []
    
    north = [row - 1, col]
    south = [row + 1, col]
    west = [row, col - 1]
    east = [row, col + 1]
             
    # North
    if north[0] >= 0 and (move_matrix[north[0], north[1]] == 4 or move_matrix[row, col] == 1):
        possible_move.append('N')
        move_position.append(north)
        
    # East
    if east[1] < maze_size[1] and (move_matrix[east[0], east[1]] == 8 or move_matrix[row, col] == 2):
        possible_move.append('E')
        move_position.append(east)
    
    # South
    if south[0] < maze_size[0] and (move_matrix[south[0], south[1]] == 1 or move_matrix[row, col] == 4):
        possible_move.append('S')
        move_position.append(south)
        
    # West
    if west[1] >= 0 and (move_matrix[west[0], west[1]] == 2 or move_matrix[row, col] == 8):
        possible_move.append('W')
        move_position.append(west)
       
    return possible_move, move_position


'''
    Function: backtrack_until()
    Description: This function backtrack the robot until the specified length of the stack
    Input:
        - env: the map for the robot
        - stack_current_path: stack containing the moves made so far
        - end_length: length of the stack after backtracking
    Output: None
'''
def backtrack_until(env, stack_current_path, end_length):
    while len(stack_current_path) != end_length:
        move = stack_current_path.pop()
        if move == 'E':
            render_path(env, 'W')
        elif move == 'W':
            render_path(env, 'E')
        elif move == 'N':
            render_path(env, 'S')
        elif move == 'S':
            render_path(env, 'N')

In [3]:
# List of possible maze id: https://github.com/MattChanTK/gym-maze/blob/master/gym_maze/__init__.py 
maze_id = 'maze-sample-10x10-v0'
env = gym.make(maze_id)
env.reset()

maze_size = env.env.maze_size
starting_point = env.env.maze_view.entrance
ending_point = env.env.maze_view.goal
col, row = env.env.state

_ = env.render()
print("Maze size = ", maze_size)
print("Starting point for the maze = ", starting_point)
print("Ending point for the maze = ", ending_point)

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.6.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Maze size =  (10, 10)
Starting point for the maze =  [0 0]
Ending point for the maze =  [9 9]


In [4]:
# Documentation for stack: https://docs.python.org/2/library/collections.html 
stack = deque()
stack_current_path = deque()   # stack to keep track of current path

visited = np.zeros(maze_size)  # keep track of visited spots

depth = 1
move = ''
stack.append([starting_point, move, depth])

while len(stack) != 0:
    current_position, move, depth = stack.pop()


    ## When the depth of the top of the stack is less that then current depth, backtrack
    if len(stack_current_path)  >= depth:

        backtrack_until(env, stack_current_path, depth - 1)
        
    stack_current_path.append(move)
    render_path(env, move, delay = 0.3)

    
    ## Found the end, exit the loop
    if current_position[0] == ending_point[0] and current_position[1] == ending_point[1]:
        break
        
    ## Mark the current spot as visited and move the next spot
    visited[current_position[0], current_position[1]] = 1
    possible_moves, move_position = get_possible_moves(env, current_position)


    ## Go through the list of next possible moves
    for i in range(len(possible_moves)):
        position = move_position[i]
        move = possible_moves[i]
        if visited[position[0], position[1]] == 0:      ## Only append if the spot hasn't been visited yet
            visited[position[0], position[1]] = 1
            stack.append([position, move, depth + 1])
            
